# 参数访问

In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand((2, 4))
net(X)

tensor([[0.3362],
        [0.3630]], grad_fn=<AddmmBackward0>)

In [4]:
"""
net 是一个 nn sequential的东西 可以简单理解为一个python 的 lis
"""

print(net[2].state_dict()) # 所以net[2] 很好理解了吧

# 注意输出的OrderedDict，所以这个state_dict就是_modules那个东西

OrderedDict([('weight', tensor([[ 0.1692,  0.1239,  0.2739, -0.0479,  0.1043,  0.1554,  0.0326, -0.3394]])), ('bias', tensor([0.3505]))])


# 目标参数

In [5]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
""" 注意Parameter，是一个可以优化的参数
为什么是.data，因为还有一个梯度.grad

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3505], requires_grad=True)
tensor([0.3505])


In [6]:
net[2].weight.grad == None
# 还没有做反向计算，所以grad是None

True

# 一次性访问所有参数

In [8]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [13]:
# 那个*号在干啥，看看不用会怎么样
print([(name, param.shape) for name, param in net[0].named_parameters()])
""" 额，，，先看几个概念
元组：tuple 就这 ( )
列表：list 就这个[ ]
元组列表：[(),(),()...]懂了吧 装着多个元组的列表
然后*就是给元组列表解包，去掉最外面的[]让他们变成一个一个的参数给print
到这应该就没什么问题了

但是又发现 给[] 和不给[] 会不一样 为什么呢？"""
print([(name, param.shape) for name, param in net[0].named_parameters()])
print((name, param.shape) for name, param in net[0].named_parameters())

"""怎么说呢
列表推导式（使用 []）：
作用：立即生成一个包含所有 (name, param.shape) 元组的列表。
输出：完整的列表内容，例如 [(name1, shape1), (name2, shape2), ...]。

生成器表达式（不使用 []）：
作用：创建一个生成器对象，不立即生成所有元素。
输出：生成器的内存地址，例如 <generator object <genexpr> at 0x...>，不会显示实际内容。

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
<generator object <genexpr> at 0x000001660ABF8EB0>


In [15]:
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 没太多可以说的，注意relu是没有参数的

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [16]:
#得到名字后可以
net.state_dict()['2.bias'].data

tensor([0.3505])

# 从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8),
                         nn.ReLU(),
                         nn.Linear(8, 4),
                         nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1()) # 插入四个block1， 用add_module方法，
                # （接上）和sequential唯一的区别是，第一个参数可以传一个名字，这样就不用告诉你是1234了
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)


tensor([[-0.1047],
        [-0.1047]], grad_fn=<AddmmBackward0>)

In [19]:
print(rgnet)
""" 最外层是一个Sequential
然后第一个也是Sequential
然后block 0 也是一个 Sequential，三个Sequential的嵌套哈哈

复杂的网络print就没那么方便

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# 内置初始化

In [20]:
def init_normal(m): # m就是一个module 每次传入一个module
    if type(m) == nn.Linear:
        # !!! 下划线在后面，表示一个替换函数，表示不会返回一个值，而是直接替换掉
        nn.init.normal_(m.weight, mean=0, std=0.01) 
        nn.init.zeros_(m.bias)

net.apply(init_normal) # 遍历 apply给
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0171, -0.0007, -0.0059, -0.0018]), tensor(0.))

In [22]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) # 从算法角度 不能这样做， 但是
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [24]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0420,  0.6849, -0.4352, -0.2633])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
# why 一个取[0] 一个不取

In [25]:
print(net[2].weight.data[0])

tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [26]:
print(net[0].weight.data)

tensor([[ 0.0420,  0.6849, -0.4352, -0.2633],
        [-0.1014, -0.6716, -0.5937,  0.4508],
        [ 0.3130, -0.3959,  0.3589,  0.1162],
        [-0.1730,  0.2663,  0.2891, -0.0538],
        [-0.4665, -0.4108, -0.5707,  0.5022],
        [ 0.2139,  0.0316, -0.0763,  0.2547],
        [ 0.0616, -0.1022, -0.4456, -0.2473],
        [-0.6715, -0.3633, -0.2692,  0.3516]])


In [27]:
net[0].weight

Parameter containing:
tensor([[ 0.0420,  0.6849, -0.4352, -0.2633],
        [-0.1014, -0.6716, -0.5937,  0.4508],
        [ 0.3130, -0.3959,  0.3589,  0.1162],
        [-0.1730,  0.2663,  0.2891, -0.0538],
        [-0.4665, -0.4108, -0.5707,  0.5022],
        [ 0.2139,  0.0316, -0.0763,  0.2547],
        [ 0.0616, -0.1022, -0.4456, -0.2473],
        [-0.6715, -0.3633, -0.2692,  0.3516]], requires_grad=True)

In [28]:
net[0].weight.shape

torch.Size([8, 4])

In [30]:
net[2].weight.shape

torch.Size([1, 8])

In [ ]:
# 20241201 2059 好像是按相反的来 为什么会这样呢 我记得linear里面是点积 吧？ 那乘的时候weight是否转置了呢

https://blog.csdn.net/zhaohongfei_358/article/details/122797190

讲的非常非常好！！！

# 自定义初始化

In [31]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))


tensor([[ 0.0000,  0.0000,  0.0000,  9.5448],
        [ 0.0000,  9.1330,  5.0017, -8.4905]], grad_fn=<SliceBackward0>)

In [32]:
net[0].weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  9.5448],
        [ 0.0000,  9.1330,  5.0017, -8.4905],
        [-8.2355,  8.7172, -5.4528,  0.0000],
        [-0.0000, -0.0000,  7.0884, -0.0000],
        [ 8.0613, -6.5059, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -7.0752,  9.0798],
        [ 0.0000,  0.0000, -8.4002, -0.0000],
        [ 0.0000,  0.0000, -9.3272,  9.9640]], requires_grad=True)

In [33]:
# 更简单 直接暴力

In [38]:
net[0].weight.data[:] += 1

In [39]:
net[0].weight.data[0, 0] = 42

In [40]:
net[0].weight.data[0]

tensor([42.0000,  3.0000,  3.0000, 12.5448])

# 参数绑定

In [42]:
# 两个数据流进来 ， 共享权重 ， 共享就是一样的意思， 

In [44]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    shared,
                    nn.ReLU(),
                    nn.Linear(8,1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
# 每个层之间去共享参数 （感觉从编程语言的角度去理解比较好，但李沐没那样讲）

net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [50]:
net[0].weight.data.shape

torch.Size([8, 4])